## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-11-10-22-09 +0000,nypost,Luigi Mangione’s lawyers seek dismissal of fed...,https://nypost.com/2025/10/11/us-news/luigi-ma...
1,2025-10-11-10-03-32 +0000,nyt,Following Through,https://www.nytimes.com/2025/10/11/briefing/fo...
2,2025-10-11-10-00-47 +0000,wapo,Trump administration says immigration enforcem...,https://www.washingtonpost.com/business/2025/1...
3,2025-10-11-10-00-06 +0000,nyt,"A New Book Asks, What Is Canada?",https://www.nytimes.com/2025/10/11/world/canad...
4,2025-10-11-10-00-00 +0000,latimes,"As hydrants ran dry, LAFD crews faced delays g...",https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2335/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,57
835,nobel,20
19,new,19
189,china,17
868,peace,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
298,2025-10-10-14-32-26 +0000,nypost,"Fresh off Nobel Peace Prize win, opposition le...",https://nypost.com/2025/10/10/world-news/nobel...,166
136,2025-10-10-22-46-56 +0000,nyt,María Corina Machado Wins Nobel Peace Prize De...,https://www.nytimes.com/2025/10/10/world/ameri...,147
241,2025-10-10-17-17-55 +0000,nypost,Nobel Peace Prize winner Maria Corina Machado ...,https://nypost.com/2025/10/10/us-news/nobel-pe...,139
217,2025-10-10-18-33-53 +0000,bbc,As Nobel Prize goes to Venezuela's opposition ...,https://www.bbc.com/news/articles/crmegp9lkjro...,127
178,2025-10-10-21-03-30 +0000,bbc,White House blasts Nobel Committee for not awa...,https://www.bbc.com/news/articles/c7842qg15p6o...,125


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
298,166,2025-10-10-14-32-26 +0000,nypost,"Fresh off Nobel Peace Prize win, opposition le...",https://nypost.com/2025/10/10/world-news/nobel...
173,69,2025-10-10-21-13-00 +0000,wsj,"House Speaker Mike Johnson (R., La.) received ...",https://www.wsj.com/politics/policy/mike-johns...
79,58,2025-10-11-02-31-03 +0000,nypost,NY AG Letitia James admitted Virginia home at ...,https://nypost.com/2025/10/10/us-news/ny-ag-le...
141,52,2025-10-10-22-34-17 +0000,nypost,Trump gives new details on Mideast trip as Ham...,https://nypost.com/2025/10/10/us-news/trump-gi...
294,41,2025-10-10-14-45-15 +0000,latimes,"Sister Jean, nun who became a national icon du...",https://www.latimes.com/sports/story/2025-10-1...
263,41,2025-10-10-16-11-00 +0000,wsj,The Trump administration is recalling some fur...,https://www.wsj.com/economy/cpi-inflation-repo...
166,41,2025-10-10-21-36-00 +0000,wsj,Trump’s Threat on Higher China Tariffs Wipes O...,https://www.wsj.com/finance/stocks/trumps-thre...
100,40,2025-10-11-00-46-45 +0000,wapo,19 missing after blast rocked Tennessee milita...,https://www.washingtonpost.com/nation/2025/10/...
266,39,2025-10-10-16-02-34 +0000,bbc,No plans to send UK troops to monitor Gaza cea...,https://www.bbc.com/news/articles/cx2pek0lz6mo...
123,36,2025-10-10-23-24-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
